In [53]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from QBUS2820 import rmse_jack, r2_jack 
from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from ExtraCode import getResultTable
from sklearn.model_selection import cross_val_score
from QBUS2820 import forward
from sklearn.linear_model import ElasticNet

### It's ok...

In [61]:
pred = []
method = []

data = pd.read_csv('Train6_1.csv')
final_train = data.sample(frac=0.6, random_state=1)
final_test = data[data.index.isin(final_train.index)==False]
final_train.head()
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')
y_train = np.log(y_train)


#Forward selection
fwd = forward()
fwd.fit(final_train, y_train)
predforward = fwd.predict(final_test)
method.append('Forward')
pred.append(np.exp(predforward))

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(final_train,y_train)
predFinalBoost = GBoost.predict(final_test)
method.append('GBoost')
pred.append(np.exp(predFinalBoost))

regr = AdaBoostRegressor(loss='linear', learning_rate = 1, n_estimators = 350)
regr = regr.fit(final_train,y_train)
adapred = regr.predict(final_test)
method.append('AdaBoost')
pred.append(np.exp(adapred))

In [62]:
xgdmat = xgb.DMatrix(final_train, y_train) # Create our DMatrix to make XGBoost more efficient
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
testFinaldmat = xgb.DMatrix(final_test)
xgpred = final_gb.predict(testFinaldmat)
method.append('XGBoost')
pred.append(np.exp(xgpred))

In [63]:
pred2 = (np.exp(xgpred)+np.exp(adapred)+np.exp(predFinalBoost))/3
pred.append(pred2)
method.append('XGBoost, ada, GB')

In [64]:
pred3 = (np.exp(xgpred)+np.exp(predFinalBoost))/2

pred5 = (np.exp(xgpred)+np.exp(adapred))/2
pred.append(pred3)
pred.append(pred5)
method.append('xg GB')
method.append('xg adaboost')

In [65]:
#Extremely Random forest
regr2 = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr2 = regr.fit(final_train,y_train)

In [66]:

predFinalExtRandomForestlad = regr.predict(final_test)

pred5 = (np.exp(xgpred)+np.exp(predFinalExtRandomForestlad)+np.exp(predFinalBoost))/3
pred.append(pred5)
method.append('XGBoost, random tree, GB')

In [67]:
pred6 = (np.exp(xgpred)+np.exp(predFinalExtRandomForestlad))/2
pred.append(pred6)
method.append('xg Random tree')

In [68]:
pred10 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad))/3
pred.append(pred10)
method.append('XGBoost, GBoost Random Forest')

In [69]:
#Validation function
n_folds = 10
def RMSLE(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(final_train.values)
    rmse= np.sqrt(-cross_val_score(model, final_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return rmse

In [70]:
#LASSO
lasso = Lasso(alpha = 0.1)
lasso.fit(final_train, y_train) 
pred_L = lasso.predict(final_test)
method.append('LASSO_Alpha_zeropointone')
pred.append(np.exp(pred_L))



/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [25]:

alphas = [0.1, 1, 5, 10, 20, 50, 75, 100]
for alpha in alphas:
    rg = Lasso(alpha = alpha)
    rg.fit(final_train, y_train)
    rg_score = rg.score(final_train, y_train)
    rmse = RMSLE(rg).mean()
    print("For alpha = {} the Lasso model accuracy score = {:.2f} and  mean rmse = {:.3f}".format(alpha, rg_score*100, rmse))

/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:4

For alpha = 0.1 the Lasso model accuracy score = 96.55 and  mean rmse = 21741.100


/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:4

For alpha = 1 the Lasso model accuracy score = 96.55 and  mean rmse = 21457.697


/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:4

...

/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:4

For alpha = 75 the Lasso model accuracy score = 95.75 and  mean rmse = 18496.744


/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:4

For alpha = 100 the Lasso model accuracy score = 95.58 and  mean rmse = 18508.492


/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [11]:
#RIDGE
#alphas = np.exp(np.linspace(-10,20,500)) 
#ridge = RidgeCV(alphas=alphas, cv=10)
#ridge.fit(final_train, y_train)
#ridge = Ridge(alpha=ridge.alpha_)
#ridge.fit(final_train, y_train)
#pred_R = ridge.predict(final_test)
#method.append('Ridge')
#pred.append(pred_R)

alphas = [0.001, 0.01, 0.1, 1, 5, 10, 20, 50, 75, 100]
for alpha in alphas:
    rg = Ridge(alpha = alpha)
    rg.fit(final_train, y_train)
    rg_score = rg.score(final_train, y_train)
    rmse = RMSLE(rg).mean()
    print("For alpha = {} the Ridge model accuracy score = {:.2f} and  mean rmse = {:.3f}".format(alpha, rg_score*100, rmse))

For alpha = 0.001 the Ridge model accuracy score = 96.55 and  mean rmse = 21717.670
For alpha = 0.01 the Ridge model accuracy score = 96.55 and  mean rmse = 21574.424
For alpha = 0.1 the Ridge model accuracy score = 96.53 and  mean rmse = 20767.142
For alpha = 1 the Ridge model accuracy score = 96.31 and  mean rmse = 19317.776
For alpha = 5 the Ridge model accuracy score = 95.78 and  mean rmse = 18998.876
For alpha = 10 the Ridge model accuracy score = 95.40 and  mean rmse = 19176.510
For alpha = 20 the Ridge model accuracy score = 94.92 and  mean rmse = 19516.232
For alpha = 50 the Ridge model accuracy score = 94.14 and  mean rmse = 20174.935
For alpha = 75 the Ridge model accuracy score = 93.74 and  mean rmse = 20549.666
For alpha = 100 the Ridge model accuracy score = 93.44 and  mean rmse = 20853.185


In [71]:
alpha = 1
rg = Ridge(alpha = alpha)
rg.fit(final_train, y_train)
y_pred_rg = (rg.predict(final_test))
y_pred_rg

pred.append(np.exp(y_pred_rg))
method.append('Ridge')

In [72]:
pred11 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(y_pred_rg))/4
pred.append(pred11)
method.append('XGBoost, GBoost Random Forest Ridge')

In [73]:
pred12 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(pred_L))/4
pred.append(pred12)
method.append('XGBoost, GBoost Random Forest LASSO')

In [74]:
from sklearn.linear_model import ElasticNet

regr = ElasticNet(random_state=0, alpha = 1)
    
regr.fit(final_train, y_train)    
pred13 = regr.predict(final_test)
pred.append(np.exp(pred13))
method.append('ENET')

In [75]:
pred12 = (xgpred+predFinalBoost+predFinalExtRandomForestlad+pred13)/4
pred.append(pred12)
method.append('XGBoost, GBoost Random Forest ENET')

In [76]:
pred13 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg))/3
pred.append(pred13)
method.append('XGBoost, GBoost Ridge')

In [78]:
getResultTable(method,pred)

,Test RMSE,SE,Jack R2,SE,MAE,R-square
Forward,23201.422,4765.336,0.890,0.044,13758.475,0.890
GBoost,19919.416,2775.611,0.919,0.014,13403.900,0.919
AdaBoost,25885.262,3635.328,0.863,0.024,18109.292,0.863
XGBoost,20712.606,2993.123,0.912,0.017,13922.622,0.912
"XGBoost, ada, GB",20611.238,3379.663,0.913,0.019,13849.615,0.913
xg GB,19709.564,2972.069,0.921,0.016,13248.823,0.921
xg adaboost,21820.714,3545.629,0.903,0.021,14830.583,0.903
"XGBoost, random tree, GB",20650.433,3418.845,0.913,0.019,13844.993,0.913
xg Random tree,21892.704,3600.970,0.902,0.022,14821.992,0.902
"XGBoost, GBoost Random Forest",20650.433,3418.845,0.913,0.019,13844.993,0.913


In [77]:
def getResultTable(rows, predictions):
    columns=['Test RMSE', 'SE', 'Jack R2', 'SE', 'MAE', 'R-square']
    results=pd.DataFrame(0.0, columns=columns, index=rows)

    for row,pred in zip(range(0,len(rows)),predictions):
        results.iloc[row,0], results.iloc[row,1] = rmse_jack(y_test, pred)
        results.iloc[row,2], results.iloc[row,3] = (r2_jack(y_test, pred))
        results.iloc[row,4] = mean_absolute_error(y_test, pred)
        results.iloc[row,5] = r2_score(y_test,pred)
    return results.round(3)

# Kaggle Predictions

In [21]:
data = pd.read_csv('Train6_1.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_1.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [22]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
prediction = final_gb.predict(testFinaldmat)

In [23]:
#Extremely Random forest
regr = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr = regr.fit(data,y_price)
predFinalExtRandomForestlad = regr.predict(kaggle)

In [24]:

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [25]:
alpha = 1
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [26]:
finalprediction = (np.exp(predFinalBoost)+np.exp(prediction)+np.exp(predFinalExtRandomForestlad)+np.exp(y_pred_rg))/4

In [27]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction5 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction5.to_csv("Day6_3.csv", index=False)

# Best model is just XGboost + Boosting Tree

# Allowing for model stacking now

### 

In [23]:
data = pd.read_csv('TrainA.csv')
y_price = data.pop('LogPrice')
kaggle = pd.read_csv('TestA.csv')
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [24]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
prediction = final_gb.predict(testFinaldmat)

In [25]:
#Extremely Random forest
regr = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr = regr.fit(data,y_price)
predFinalExtRandomForestlad = regr.predict(kaggle)

In [26]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [27]:
alpha = 1
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [28]:
finalprediction = (np.exp(predFinalBoost)+np.exp(prediction)+np.exp(predFinalExtRandomForestlad)+np.exp(y_pred_rg))/4

In [29]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction5 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction5.to_csv("Day6_4.csv", index=False)

## Does worse

In [79]:
data = pd.read_csv('Train6_1.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_1.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [80]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
prediction = final_gb.predict(testFinaldmat)

In [81]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [82]:
alpha = 1
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [84]:
finalprediction = (np.exp(predFinalBoost)+np.exp(prediction)+np.exp(y_pred_rg))/3

In [85]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction5 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction5.to_csv("Day6_5.csv", index=False)